In [22]:
import base64
import json
import os
import requests
import pandas as pd
from prophet import Prophet
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from utils import upload
import dropbox
import time
import datetime

load_dotenv()

def upload_to_dropbox(dbx, fullname, folder, subfolder, name, overwrite=False):
    """Upload a file.
    Return the request response, or None in case of error.
    """
    path = '/%s/%s/%s' % (folder, subfolder.replace(os.path.sep, '/'), name)
    while '//' in path:
        path = path.replace('//', '/')
    mode = (dropbox.files.WriteMode.overwrite
            if overwrite
            else dropbox.files.WriteMode.add)
    mtime = os.path.getmtime(fullname)
    with open(fullname, 'rb') as f:
        data = f.read()
    try:
        res = dbx.files_upload(
            data, path, mode,
            client_modified=datetime.datetime(*time.gmtime(mtime)[:6]),
            mute=True)
        check_for_links = dbx.sharing_list_shared_links(path)
        if check_for_links.links:
            link_to_file = check_for_links.links[0].url
        else:
            shared_link_metadata = dbx.sharing_create_shared_link_with_settings(path)
            link_to_file = shared_link_metadata.url

    except dropbox.exceptions.ApiError as err:
        print('*** API error', err)
        return None
    print('uploaded as', res.name.encode('utf8'))
    return link_to_file

allData = pd.read_csv(
    'https://www.dropbox.com/scl/fi/ksf0nbmmiort5khbrgr61/allData.csv?rlkey=75e735fjk4ifttjt553ukxt3k&dl=1')
allData.ds = pd.to_datetime(allData.ds)

df = allData.copy()
df.ds = pd.to_datetime(df.ds)

df['total_tbs'] = df[['TRG_HALLWAY_TBS',
                      'POD_GREEN_TBS',
                      'POD_YELLOW_TBS',
                      'POD_ORANGE_TBS',
                      'RAZ_TBS',
                      'AMBVERTTBS',
                      'QTrack_TBS',
                      'Garage_TBS']].sum(axis=1)
df['vert_tbs'] = df[[
    'RAZ_TBS',
    'AMBVERTTBS',
    'QTrack_TBS',
    'Garage_TBS']].sum(axis=1)
df['pod_tbs'] = df[['TRG_HALLWAY_TBS',
                    'POD_GREEN_TBS',
                    'POD_YELLOW_TBS',
                    'POD_ORANGE_TBS',
                    ]].sum(axis=1)
df.tail()

tbs_columns = ['total_tbs', 'vert_tbs', 'pod_tbs']

output = pd.DataFrame()
FIRST_RUN = True

column = 'total_tbs'

try:
    print('working on '+column)

    m = Prophet(interval_width=0.95)
    m.fit(df[['ds', column]].rename(columns={column: 'y'}))
    future = m.make_future_dataframe(periods=24*1, freq='h')
    forecast = m.predict(future)
    if FIRST_RUN:
        output['ds'] = forecast['ds']

    for forecast_column in ['yhat', 'yhat_lower', 'yhat_upper']:
        kwargs = {column+'_'+forecast_column: forecast[forecast_column]}
        output = output.assign(**kwargs)
    FIRST_RUN = False
except:
    print(column + ' failed')

data = df.copy()


working on total_tbs


23:04:20 - cmdstanpy - INFO - Chain [1] start processing
23:04:34 - cmdstanpy - INFO - Chain [1] done processing


,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,Garage_TBS,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs
0,2021-01-01 01:00:00,1,1,1,1,2,2,0,0,0,...,0.0,1,0,0,0,3,3,0.0,0.0,0
1,2021-01-01 02:00:00,2,3,1,2,3,5,1,1,0,...,0.0,1,1,0,0,3,3,3.0,2.0,1
2,2021-01-01 03:00:00,0,3,1,3,1,6,0,1,0,...,0.0,0,0,0,0,3,3,2.0,1.0,1
3,2021-01-01 04:00:00,1,4,0,3,1,7,1,2,0,...,0.0,0,0,0,0,4,3,1.0,0.0,1
4,2021-01-01 05:00:00,1,5,1,4,2,9,1,3,0,...,0.0,0,0,0,0,4,3,2.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35694,2025-01-29 18:00:00,8,107,2,73,10,180,0,28,1,...,0.0,7,0,0,0,9,8,27.0,17.0,10
35695,2025-01-29 19:00:00,9,116,4,77,13,193,2,30,0,...,0.0,10,0,0,0,9,8,27.0,19.0,8
35696,2025-01-29 20:00:00,9,125,5,82,14,207,3,33,0,...,0.0,13,1,0,0,10,8,29.0,17.0,12
35697,2025-01-29 21:00:00,9,134,7,89,16,223,4,37,0,...,0.0,12,1,0,0,10,9,31.0,14.0,17


In [23]:
output

,ds,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper
0,2021-01-01 01:00:00,2.411489,-13.560335,18.105654
1,2021-01-01 02:00:00,1.314207,-14.986571,18.166957
2,2021-01-01 03:00:00,-1.018284,-17.031891,15.658596
3,2021-01-01 04:00:00,-4.104736,-19.147601,12.279305
4,2021-01-01 05:00:00,-6.678119,-24.633938,10.381231
...,...,...,...,...
35718,2025-01-30 18:00:00,28.842259,13.051263,45.437380
35719,2025-01-30 19:00:00,26.618548,10.823732,42.223445
35720,2025-01-30 20:00:00,24.830276,8.769146,40.651345
35721,2025-01-30 21:00:00,23.291878,7.652869,40.448317


In [24]:
total_tbs = df[['ds','total_tbs']]
total_tbs_forecast = output[['ds', 'total_tbs_yhat', 'total_tbs_yhat_lower', 'total_tbs_yhat_upper']]

merged_df = pd.merge(
    total_tbs, 
    total_tbs_forecast, 
    on='ds', 
    how='outer'
)

merged_df

,ds,total_tbs,total_tbs_yhat,total_tbs_yhat_lower,total_tbs_yhat_upper
0,2021-01-01 01:00:00,0.0,2.411489,-13.560335,18.105654
1,2021-01-01 02:00:00,3.0,1.314207,-14.986571,18.166957
2,2021-01-01 03:00:00,2.0,-1.018284,-17.031891,15.658596
3,2021-01-01 04:00:00,1.0,-4.104736,-19.147601,12.279305
4,2021-01-01 05:00:00,2.0,-6.678119,-24.633938,10.381231
...,...,...,...,...,...
35718,2025-01-30 18:00:00,NaN,28.842259,13.051263,45.437380
35719,2025-01-30 19:00:00,NaN,26.618548,10.823732,42.223445
35720,2025-01-30 20:00:00,NaN,24.830276,8.769146,40.651345
35721,2025-01-30 21:00:00,NaN,23.291878,7.652869,40.448317


In [25]:
# Create an Excel writer using XlsxWriter
file_name = "total_tbs.xlsx"
with pd.ExcelWriter(file_name, engine='xlsxwriter', datetime_format="yyyy-mm-dd hh:mm:ss") as writer:
    merged_df.to_excel(writer, sheet_name='Sheet1', index=False)

    # Get the workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Define the range for the table
    num_rows, num_cols = merged_df.shape
    col_letters = [chr(65 + i) for i in range(num_cols)]  # Column letters (A, B, C...)
    table_range = f"A1:{col_letters[-1]}{num_rows+1}"  # Adjusting for headers

    # Add an Excel table
    worksheet.add_table(table_range, {
        'columns': [{'header': col} for col in merged_df.columns],
        'name': 'total_tbs',  # Optional: Name your table
        'style': 'Table Style Medium 9'  # Choose a predefined style
    })

    # Save the file
    writer.close()

/Users/devinhopkins/anaconda3/envs/ed-flow-2023/lib/python3.10/site-packages/xlsxwriter/workbook.py:377: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
